In [5]:
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm


# Implementing the DGP

In [95]:
N = 100
T = 6  # T = 5 in the assignment but since we also included the index 0, we actually have T = 6
rho = 0.5
pi = 1
beta = 1
theta = 0
np.random.seed(2)
def DGP (N, T, rho, pi, beta, theta):
    alpha = np.random.normal(size = N)
    zeta = np.random.normal(size = N)
    nu = np.random.normal(size = N)
    eps = np.random.normal(size = N)
    x0 = pi*alpha /(1-rho) + (theta*nu + zeta)/np.sqrt(1-rho**2)
    y0 = beta*x0 + alpha + eps
    x = np.array([x0])
    y = np.array([y0])
    for i in range(1,T):
        xi = np.random.normal(size = N)
        xt = rho*x[i-1] + pi*alpha + theta*eps +xi
        x = np.append(x, [xt],axis=0)
        eps = np.random.normal(size = N)
        yt = beta*xt + alpha + eps
        y = np.append(y, [yt], axis=0)
    x = x.transpose()
    y = y.transpose()
    return(y, x)

def PooledOLS(y, x):
    y_pool = y.ravel()
    x_pool = x.ravel()
    model = sm.OLS(y_pool, x_pool)
    results = model.fit()
    return([results,x_pool,y_pool])

def FD(y, x):
    dy = np.diff(y)
    dx = np.diff(x)
    dy_pool = dy.ravel()
    dx_pool = dx.ravel()
    model = sm.OLS(dy_pool, dx_pool)
    results = model.fit()
    return([results, dx_pool, dy_pool])

def FE(y, x):
    y_FE = (y - y.mean(axis=1, keepdims=True)).ravel()
    x_FE = (x - x.mean(axis=1, keepdims=True)).ravel()
    model = sm.OLS(y_FE, x_FE)
    results = model.fit()
    return([results, x_FE, y_FE])

def GmmIV(y, x, Z):   # We implemented the IV estimator based on the page 745 of Cameron & Trivedi, because we could not find
    y = np.delete(y,0,1)              # the proper package.
    x = np.delete(x,0,1)
    y = np.diff(y)
    x = np.diff(x)
    XZ, ZX, ZY, W = 0, 0, 0, 0
    T = len(y[0])
    N = len(y)
    D = np.zeros([T,T+1])
    for i in range(T):
        D[i][i] = -1
        D[i][i+1] = 1
    for i in range(N):
        XZ = XZ + x[i].dot(Z[i])
        ZX = ZX + Z[i].transpose().dot(x[i])
        ZY = ZY + Z[i].transpose().dot(y[i])
        W = W + Z[i].transpose().dot(D).dot(D.transpose()).dot(Z[i])
    if Z[0].shape == (len(Z[0]),):   # some if and else statements are needed because .dot and linalg does not work with scalar 
        W = 1 / W
        Inv = XZ * W * ZX
        Inv = 1/Inv
        Q = XZ * W * ZY
        IV = Inv * Q
    else:
        W = np.linalg.inv(W)
        Inv = XZ.dot(W).dot(ZX)
        if Inv.shape == ():
            Inv = 1/Inv
            Q = XZ.dot(W).dot(ZY)
            IV = Inv * Q
        else:
            Inv = np.linalg.inv(Inv)
            Q = XZ.dot(W).dot(ZY)
            IV = Inv.dot(Q)
    return([IV,x,y])

#  Exercise 1
We first performed Pooled OLS (POLS), FD-OLS and FE-OLS with 1000 simulations. We expect that Pooled OLS is inconsistent, since $x_{it}$ and $\alpha_i$ are correlated. Therefore we also expect that the POLS to be biased and the t-test is also biased (with a biased t-test of an estimator we mean that the t-test rejects the null-hypothesis H0 : $\beta = 1$, and hence it is not a good test).\
We do expect that FD-OLS and FE_OLS are consistent and not biased, since these estimators eliminated $\alpha_i$. Furthermore, we have $\theta=0$. Therefore, the regressors is not correlated with the error.\
The simulations below confirmed our expectations.

In [96]:
M = 1000
beta_est = np.zeros([3,M])
beta_cov = np.zeros([3,M])
accepted = np.zeros(3)
for n in range(M):
    [y, x] = DGP(N, T, rho, pi, beta, theta)
    result = PooledOLS(y, x)[0]
    beta_est[0][n] = result.params[0]
    beta_cov[0][n] = result.bse[0]
    if result.t_test((1,1)).conf_int()[0][0] <= 1 and result.t_test((1,1)).conf_int()[0][1] >= 1:
        accepted[0] += 1
    result = FD(y, x)[0]
    beta_est[1][n] = result.params[0]
    beta_cov[1][n] = result.bse[0]
    if result.t_test((1,1)).conf_int()[0][0] <= 1 and result.t_test((1,1)).conf_int()[0][1] >= 1:
        accepted[1] += 1
    result = FE(y, x)[0]
    beta_est[2][n] = result.params[0]
    beta_cov[2][n] = result.bse[0]
    if result.t_test((1,1)).conf_int()[0][0] <= 1 and result.t_test((1,1)).conf_int()[0][1] >= 1:
        accepted[2] += 1

In [97]:
print('The mean of the estimators of respectively POLS, FD and FE:')
print(np.mean(beta_est, axis = 1))
print('The average of the standard deviation of the estimators of respectively POLS, FD and FE:')
print(np.mean(beta_cov, axis = 1))
print('The frequency of not rejecting the nul-hypothesis H0: beta = 1 by the t-test of POLS, FD and FE')
print(accepted/1000)
#bins1 = np.histogram(beta_est[0])[1]
#plt.hist(beta_est[0], bins = bins1, density = True, histtype = 'bar')
#plt.show()

The mean of the estimators of respectively POLS, FD and FE:
[1.37506661 0.99775135 0.99856202]
The average of the standard deviation of the estimators of respectively POLS, FD and FE:
[0.01982971 0.05481979 0.04140601]
The frequency of not rejecting the nul-hypothesis H0: beta = 1 by the t-test of POLS, FD and FE
[0.    0.914 0.92 ]


# Exercise 2

To test the robustness of FD and FE-estimators, we chose some specific values of $\pi$ that ranges from 0.001 to 1000. We expect that FD and FE are robust to changes of $\pi$, since they both eliminated $\alpha_i$ when differencing. The simulation below indeed shows that both estimators are still unbiased.  

In [98]:
M = 1000
pi_values = np.array([0.001, 0.1, 5, 50, 1000]) 
beta_FD = np.zeros([len(pi_values), M])
beta_FDsd = np.zeros([len(pi_values), M])
beta_FE = np.zeros([len(pi_values), M])
beta_FEsd = np.zeros([len(pi_values), M])
i = 0
for p in pi_values:
    for n in range(M):
        [y, x] = DGP(N, T, rho, p, beta, theta)
        result = FD(y, x)[0]
        beta_FD[i][n] = result.params[0]
        beta_FDsd[i][n] = result.bse[0]
        result = FE(y, x)[0]
        beta_FE[i][n] = result.params[0]
        beta_FEsd[i][n] = result.bse[0]
    i += 1
       

In [99]:
print('The mean of the estimator of FD over 5 values of pi:')
print(np.mean(beta_FD, axis = 1, keepdims = True))
print('The average of the standard deviation of the estimator FD over 5 values of pi:')
print(np.mean(beta_FDsd, axis = 1, keepdims = True))
print('The mean of the estimator of FE over 5 values of pi:')
print(np.mean(beta_FE, axis = 1, keepdims = True))
print('The average of the standard deviation of the estimator FE over 5 values of pi:')
print(np.mean(beta_FEsd, axis = 1, keepdims = True))

The mean of the estimator of FD over 5 values of pi:
[[1.00155294]
 [1.00190575]
 [0.99938626]
 [1.00182775]
 [0.99989678]]
The average of the standard deviation of the estimator FD over 5 values of pi:
[[0.05482207]
 [0.05488792]
 [0.05479792]
 [0.05485425]
 [0.05490479]]
The mean of the estimator of FE over 5 values of pi:
[[1.0010828 ]
 [1.00087461]
 [0.99850841]
 [1.00132603]
 [1.00103798]]
The average of the standard deviation of the estimator FE over 5 values of pi:
[[0.04135248]
 [0.04144228]
 [0.04142116]
 [0.04153766]
 [0.04147161]]


# Exercise 3
We analyzed the impact of $\theta = 0.5$ for the FD and FE estimators. Since $\theta\neq 0$, we now have that $x_{it}$ is correlated with $\epsilon_{it-1}$. Therefore $\Delta x_{it}$ is correlated with $\Delta \epsilon_{it}$ and $x_i - 1/T \sum_t x_{it}$ is correlated with $\epsilon_i -1/T\sum_t \epsilon_{it}$. Therefore, we expect that FD and FE estimators are biased. The results of the simulation below agree with our expectation.

In [100]:
M = 1000 
beta_FD = np.zeros(M)
beta_FDsd = np.zeros(M)
beta_FE = np.zeros(M)
beta_FEsd = np.zeros(M)
theta = 0.5
for n in range(M):
    [y, x] = DGP(N, T, rho, pi, beta, theta)
    result = FD(y, x)[0]
    beta_FD[n] = result.params[0]
    beta_FDsd[n] = result.bse[0]
    result = FE(y, x)[0]
    beta_FE[n] = result.params[0]
    beta_FEsd[n] = result.bse[0]    
    

In [101]:
print('The mean of the estimator of FD for theta = 0.5:')
print(np.mean(beta_FD))
print('The average of the standard deviation of the estimator FD:')
print(np.mean(beta_FDsd))
print('The mean of the estimator of FE for theta = 5:')
print(np.mean(beta_FE))
print('The average of the standard deviation of the estimator FE:')
print(np.mean(beta_FEsd))

The mean of the estimator of FD for theta = 0.5:
0.7012540872881287
The average of the standard deviation of the estimator FD:
0.047127249694586405
The mean of the estimator of FE for theta = 5:
0.8896116765867669
The average of the standard deviation of the estimator FE:
0.036660910060674894


# Exercise 4
We impletemented and printed the $Z_i$ matrices. We are considering three instrument matrices which we denote $Z_1$, $Z_2$ and $Z_3$. They are matrices that respectively uses 1, $(T-1)$ and $1/2T(T-1)$ instruments.

In [102]:
theta = 0.5
[y, x] = DGP(N, T, rho, pi, beta, theta)
Z1 = np.zeros([N,T-2])
for i in range(N):
    for t in range((T-2)):
        Z1[i][t] = x[i][t+1]
print('The Z-matrix when we use 1 instrument:')
print(Z1)
Z2 = np.zeros([N, T-2, T-2])
for i in range(N):
    for t in range((T-2)):
        Z2[i][t][t] = x[i][t+1]
r = 1 + (T-3) * 2
print('The Z-matrix when we use T-1 instruments:')
print(Z2)
Z3 = np.zeros([N, T-2, r])
for i in range(N):
    Z3[i][0][0] = x[i][1]
    for t in range(1,(T-2)):
        Z3[i][t][t:(2*t+1)] = x[i][1:(t+2)]
print('The Z-matrix when we use 1/2T(T-1) instruments:')
print(Z3)

The Z-matrix when we use 1 instrument:
[[ 2.21535107 -0.01557787 -0.77228213 -0.23094584]
 [ 1.63046986  3.60999553  0.79243676  3.80038479]
 [ 2.00026584  2.96840115  2.89115519  3.55719188]
 [-1.56789146 -1.28296065  0.25210545  1.02508187]
 [ 1.21798656  1.91053711  2.22455612  2.37959091]
 [-0.34650266  1.08739286  0.65036611  2.8645199 ]
 [ 2.40417779  3.14835973  3.53576829  2.85063266]
 [ 1.35072159 -0.47551072  2.53208371  2.47842111]
 [-1.05250117 -0.9149518   1.58669109 -0.68628944]
 [-3.53983405 -5.14037862 -3.37118937 -5.73196952]
 [ 2.86860506  4.79682918  3.29959592  4.90200808]
 [ 0.31612718 -0.67670241  0.16830319 -1.46905925]
 [ 2.91277602  3.64572824  5.37423391  3.86224645]
 [ 2.28590472  3.01253398  1.11498655  0.3309319 ]
 [-1.80797931 -4.73875971 -1.14360492 -0.01423225]
 [ 2.29316251  0.46358194  1.26260989  1.72560399]
 [-2.69320628 -1.55791531 -0.50793143 -1.61538   ]
 [-1.79028859 -3.14514583  0.06798962  0.29246337]
 [ 2.66284266  2.70419502  1.62678984  3.25

#  Exercise 5
We looked at the bias and standard-deviation of the IV-estimator using instrument matrix Z1 (with 1 instrument), Z2 (with T-1 instruments) and Z3 (with $\frac{1}{2}T(T-1)$ instruments). We expect that these are all unbiased, since our instrument is uncorrelated with the errors. We have also varied $\theta$ and we observed that all three estimators are unbiased for a range of $\theta$ from 0.001 to 1000. We did not observe any preference for the estimators, in terms of unbiasedness.

In [103]:
M = 1000
thetas = [0.001, 0.5, 20, 1000]
beta_IV = np.zeros([len(thetas), 3, M])
j = 0
r = 1 + (T-3) * 2
Z1 = np.zeros([N,T-2])
Z2 = np.zeros([N, T-2, T-2])
Z3 = np.zeros([N, T-2, r])
for theta in thetas:
    for n in range(M):
        [y, x] = DGP(N, T, rho, pi, beta, theta)
        for i in range(N):
            for t in range((T-2)):
                Z1[i][t] = x[i][t+1]
                Z2[i][t][t] = x[i][t+1]
        for i in range(N):
            Z3[i][0][0] = x[i][1]
            for t in range(1,(T-2)):
                Z3[i][t][t:(2*t+1)] = x[i][1:(t+2)]
        beta_IV[j][0][n] = GmmIV(y, x, Z1)[0]
        beta_IV[j][1][n] = GmmIV(y, x, Z2)[0]
        beta_IV[j][2][n] = GmmIV(y, x, Z3)[0]
    j += 1


In [104]:
print('The mean of the IV-estimators, respectively Z1, Z2 and Z3, calculated for theta = 0.001')
print(np.mean(beta_IV[0], axis = 1))
print('The standard deviation of the IV-estimators, respectively Z1, Z2 and Z3, calculated for theta = 0.001')
print(np.std(beta_IV[0], axis = 1))
print('The mean of the IV-estimators, respectively Z1, Z2 and Z3, calculated for theta = 0.5')
print(np.mean(beta_IV[1], axis = 1))
print('The standard deviation of the IV-estimators, respectively Z1, Z2 and Z3, calculated for theta = 0.5')
print(np.std(beta_IV[1], axis = 1))
print('The mean of the IV-estimators, respectively Z1, Z2 and Z3, calculated for theta = 20')
print(np.mean(beta_IV[2], axis = 1))
print('The standard deviation of the IV-estimators, respectively Z1, Z2 and Z3, calculated for theta = 20')
print(np.std(beta_IV[2], axis = 1))
print('The mean of the IV-estimators, respectively Z1, Z2 and Z3, calculated for theta = 1000')
print(np.mean(beta_IV[3], axis = 1))
print('The standard deviation of the IV-estimators, respectively Z1, Z2 and Z3, calculated for theta = 1000')
print(np.std(beta_IV[3], axis = 1))

The mean of the IV-estimators, respectively Z1, Z2 and Z3, calculated for theta = 0.001
[0.99927933 0.99978592 0.9990809 ]
The standard deviation of the IV-estimators, respectively Z1, Z2 and Z3, calculated for theta = 0.001
[0.14786741 0.13708783 0.15489238]
The mean of the IV-estimators, respectively Z1, Z2 and Z3, calculated for theta = 0.5
[0.99679396 0.98833223 0.97478881]
The standard deviation of the IV-estimators, respectively Z1, Z2 and Z3, calculated for theta = 0.5
[0.12189048 0.11478969 0.13046953]
The mean of the IV-estimators, respectively Z1, Z2 and Z3, calculated for theta = 20
[1.00011015 0.99957288 0.99918854]
The standard deviation of the IV-estimators, respectively Z1, Z2 and Z3, calculated for theta = 20
[0.00483556 0.00476227 0.00408477]
The mean of the IV-estimators, respectively Z1, Z2 and Z3, calculated for theta = 1000
[1.00000508 0.99999434 0.99998195]
The standard deviation of the IV-estimators, respectively Z1, Z2 and Z3, calculated for theta = 1000
[1.0041

# Exercise 6
We changed the sample size from 100 to 1000 and we performed the above experiment with $\theta = 0.5$ over 1000 simulations.

In [105]:
M = 1000
theta = 0.5
N = 1000
beta_IV = np.zeros([3, M])
r = 1 + (T-3) * 2
Z1 = np.zeros([N,T-2])
Z2 = np.zeros([N, T-2, T-2])
Z3 = np.zeros([N, T-2, r])

for n in range(M):
    [y, x] = DGP(N, T, rho, pi, beta, theta)
    for i in range(N):
        for t in range((T-2)):
            Z1[i][t] = x[i][t+1]
            Z2[i][t][t] = x[i][t+1]
    for i in range(N):
        Z3[i][0][0] = x[i][1]
        for t in range(1,(T-2)):
            Z3[i][t][t:(2*t+1)] = x[i][1:(t+2)]
    beta_IV[0][n] = GmmIV(y, x, Z1)[0]
    beta_IV[1][n] = GmmIV(y, x, Z2)[0]
    beta_IV[2][n] = GmmIV(y, x, Z3)[0]

In [106]:
print('The mean (over 1000 samples) of the IV-estimators, respectively Z1, Z2 and Z3, calculated for theta = 0.5')
print(np.mean(beta_IV, axis = 1))
print('The standard deviation (over 1000 samples) of the IV-estimators, respectively Z1, Z2 and Z3, calculated for theta = 0.5')
print(np.std(beta_IV, axis = 1))

The mean (over 1000 samples) of the IV-estimators, respectively Z1, Z2 and Z3, calculated for theta = 0.5
[1.00067827 0.99996193 0.9985324 ]
The standard deviation (over 1000 samples) of the IV-estimators, respectively Z1, Z2 and Z3, calculated for theta = 0.5
[0.03896822 0.03715333 0.0417763 ]


Clearly, the standard deviation of the estimators are getting smaller due to the larger sample size.

# Exercise 7
To investigate persistence we set $\rho = 0.99$. Then $x_{it-1}$ because relatively large with respect to $\epsilon_{it-1}$, $\alpha_i$ and $\xi_{it}$. Since also $\rho\to 1$, this makes $x_{it}$ and $x_{it-1}$ similar and thereby induces persistance. 

In [107]:
M = 100
theta = 0.5
N = 1000
rho = 0.99
beta_IV = np.zeros([3, M])
r = 1 + (T-3) * 2
Z1 = np.zeros([N,T-2])
Z2 = np.zeros([N, T-2, T-2])
Z3 = np.zeros([N, T-2, r])
for n in range(M):
    [y, x] = DGP(N, T, rho, pi, beta, theta)
    for i in range(N):
        for t in range((T-2)):
            Z1[i][t] = x[i][t+1]
            Z2[i][t][t] = x[i][t+1]
    for i in range(N):
        Z3[i][0][0] = x[i][1]
        for t in range(1,(T-2)):
            Z3[i][t][t:(2*t+1)] = x[i][1:(t+2)]
    beta_IV[0][n] = GmmIV(y, x, Z1)[0]
    beta_IV[1][n] = GmmIV(y, x, Z2)[0]
    beta_IV[2][n] = GmmIV(y, x, Z3)[0]

In [108]:
print('The mean (over 1000 samples) of the IV-estimators, respectively Z1, Z2 and Z3, calculated for theta = 0.5')
print(np.mean(beta_IV, axis = 1))
print('The standard deviation (over 1000 samples) of the IV-estimators, respectively Z1, Z2 and Z3, calculated for theta = 0.5')
print(np.std(beta_IV, axis = 1))

The mean (over 1000 samples) of the IV-estimators, respectively Z1, Z2 and Z3, calculated for theta = 0.5
[0.12649626 0.83238908 0.74822692]
The standard deviation (over 1000 samples) of the IV-estimators, respectively Z1, Z2 and Z3, calculated for theta = 0.5
[4.29736008 0.83362386 0.44803678]


The estimators are highly biased. This shows that our instrument is weak when we have persistance. This is because persistance makes $\Delta x_{it}$ small, which makes estimating $\beta$ hard.

# Exercise 8
In this exercise we aim to estimate $\rho$. We regressed on $x_{it-1}$.

In [109]:
M = 100
N = 100
T = 6  
rho = 0.5
pi = 1
beta = 1
theta = 0
beta_pool = np.zeros([2,M])
beta_FD = np.zeros([2,M])
beta_FE = np.zeros([2,M])

for n in range(M):
    x = DGP(N, T, rho, pi, beta, theta)[1]
    Y = np.delete(x, 0,1)
    X = np.delete(x, (len(x[0])-1),1)
    resultpool = PooledOLS (Y, X)[0]
    resultFD = FD(Y, X)[0]
    resultFE = FE(Y, X)[0]
    beta_pool[0][n] = resultpool.params
    beta_pool[1][n] = resultpool.bse
    beta_FD[0][n] = resultFD.params
    beta_FD[1][n] = resultFD.bse
    beta_FE[0][n] = resultFE.params
    beta_FE[1][n] = resultFE.bse
print("The mean and the average standard deviation of the POLS estimator over 100 simulations:")
print([np.mean(beta_pool[0]), np.mean(beta_pool[1])])
print("The mean and the average standard deviation of the FD estimator over 100 simulations:")
print([np.mean(beta_FD[0]), np.mean(beta_FD[1])])
print("The mean and the average standard deviation of the FE estimator over 100 simulations:")
print([np.mean(beta_FE[0]), np.mean(beta_FE[1])])


The mean and the average standard deviation of the POLS estimator over 100 simulations:
[0.8703968678639137, 0.021861854771394632]
The mean and the average standard deviation of the FD estimator over 100 simulations:
[-0.24919912909517816, 0.04836349169687822]
The mean and the average standard deviation of the FE estimator over 100 simulations:
[0.16981040037700112, 0.04414082378295921]


We see that the estimators are highly biased. This is also explained in Cameron and Trivedi. The Pooled OLS is biased because $x_{it-1}$ is correlated with $\alpha_i$. The FD-estimator is biased because $\Delta x_{it-1}$ is correlated with $\Delta \xi_{it}$ since $x_{it-1}$ is correlated with $\xi_{it-1}$, which is also the same reason that the FE (within) estimator is biased.

If we use a large $T$ then we get a long panel data. It is stated in Cameron and Trivedi that the biasedness of the FE (within) estimator will improve.

In [110]:
M = 100
N = 100
T = 1000 
rho = 0.5
pi = 1
beta = 1
theta = 0
beta_pool = np.zeros([2,M])
beta_FD = np.zeros([2,M])
beta_FE = np.zeros([2,M])

for n in range(M):
    x = DGP(N, T, rho, pi, beta, theta)[1]
    Y = np.delete(x, 0,1)
    X = np.delete(x, (len(x[0])-1),1)
    resultpool = PooledOLS (Y, X)[0]
    resultFD = FD(Y, X)[0]
    resultFE = FE(Y, X)[0]
    beta_pool[0][n] = resultpool.params
    beta_pool[1][n] = resultpool.bse
    beta_FD[0][n] = resultFD.params
    beta_FD[1][n] = resultFD.bse
    beta_FE[0][n] = resultFE.params
    beta_FE[1][n] = resultFE.bse
print("The mean and the average standard deviation of the POLS estimator over 100 simulations:")
print([np.mean(beta_pool[0]), np.mean(beta_pool[1])])
print("The mean and the average standard deviation of the FD estimator over 100 simulations:")
print([np.mean(beta_FD[0]), np.mean(beta_FD[1])])
print("The mean and the average standard deviation of the FE estimator over 100 simulations:")
print([np.mean(beta_FE[0]), np.mean(beta_FE[1])])


The mean and the average standard deviation of the POLS estimator over 100 simulations:
[0.8743835171738656, 0.00153267333893189]
The mean and the average standard deviation of the FD estimator over 100 simulations:
[-0.25015045528972235, 0.003064817154493543]
The mean and the average standard deviation of the FE estimator over 100 simulations:
[0.4981867826194995, 0.0027432312366061605]


Indeed, we observe that the within estimator has improved, since its estimated $\rho$ is around $0.498$, which is much closer to the true value $0.5$.

Below we also increased the sample size $N$ to 1000. 

In [111]:
M = 100
N = 1000
T = 6 
rho = 0.5
pi = 1
beta = 1
theta = 0
beta_pool = np.zeros([2,M])
beta_FD = np.zeros([2,M])
beta_FE = np.zeros([2,M])

for n in range(M):
    x = DGP(N, T, rho, pi, beta, theta)[1]
    Y = np.delete(x, 0,1)
    X = np.delete(x, (len(x[0])-1),1)
    resultpool = PooledOLS (Y, X)[0]
    resultFD = FD(Y, X)[0]
    resultFE = FE(Y, X)[0]
    beta_pool[0][n] = resultpool.params
    beta_pool[1][n] = resultpool.bse
    beta_FD[0][n] = resultFD.params
    beta_FD[1][n] = resultFD.bse
    beta_FE[0][n] = resultFE.params
    beta_FE[1][n] = resultFE.bse
print("The mean and the average standard deviation of the POLS estimator over 100 simulations:")
print([np.mean(beta_pool[0]), np.mean(beta_pool[1])])
print("The mean and the average standard deviation of the FD estimator over 100 simulations:")
print([np.mean(beta_FD[0]), np.mean(beta_FD[1])])
print("The mean and the average standard deviation of the FE estimator over 100 simulations:")
print([np.mean(beta_FE[0]), np.mean(beta_FE[1])])

The mean and the average standard deviation of the POLS estimator over 100 simulations:
[0.8758107861216111, 0.00682078749479391]
The mean and the average standard deviation of the FD estimator over 100 simulations:
[-0.2484408030405424, 0.015297953487593594]
The mean and the average standard deviation of the FE estimator over 100 simulations:
[0.16923850520170988, 0.013922870995855229]


In contrast to increasing $T$, we did not observe any changes in the biasedness.